In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
import pickle

In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
df = df.replace(['Credit reporting, credit repair services, or other personal consumer reports'],'Credit reporting')
df = df.replace(['Credit card or prepaid card'],'Credit card')
df = df.replace(['Checking or savings account'],'Checking account')
df = df.replace(['Money transfer, virtual currency, or money service'],'Money service')
df = df.replace(['Vehicle loan or lease'],'Vehicle loan')
df = df.replace(['Bank account or service'],'Bank account')
df = df.replace(['Payday loan, title loan, or personal loan'],'Personal loan')

In [24]:
df = df.drop(columns = 'Unnamed: 0', axis = 1)

In [4]:
df['tags'].value_counts()

Credit reporting    364
Debt collection     160
Mortgage            100
Credit card          76
Checking account     36
Student loan         35
Money service        21
Bank account         15
Vehicle loan         14
Personal loan         9
Consumer Loan         7
Prepaid card          3
Payday loan           2
Money transfers       2
Name: tags, dtype: int64

In [5]:
val_tags = ['Credit reporting','Debt collection','Mortgage','Credit card','Checking account','Student loan','Money service','Bank account','Vehicle loan','Personal loan','Consumer Loan','Prepaid card','Payday loan','Money transfers']

In [6]:
def print_plot(index):
    example = df[['post', 'tags']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])
print_plot(10)

XXXX XXXX provided validation of debt with inaccuracies. The card acceptance certificate dated on XX/XX/XXXX to XXXX XXXX XXXX XXXX, TX XXXX was not my current address. From XX/XX/XXXX - XX/XX/XXXX, my home address was XXXX XXXX XXXX, TX XXXX and from XX/XX/XXXX to current my home address is XXXX XXXX XXXX XXXX, TX XXXX. My email address XXXX XXXX has listed for me is XXXX. This is also invalid. 

I will be filling a copy of the acceptance certificate with my police report.
Tag: Credit reporting


In [7]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['post'] = df['post'].apply(clean_text)
print_plot(10)

xxxx xxxx provided validation debt inaccuracies card acceptance certificate dated xx xx xxxx xxxx xxxx xxxx xxxx tx xxxx current address xx xx xxxx xx xx xxxx home address xxxx xxxx xxxx tx xxxx xx xx xxxx current home address xxxx xxxx xxxx xxxx tx xxxx email address xxxx xxxx listed xxxx also invalid filling copy acceptance certificate police report
Tag: Credit reporting


In [8]:
X_val = df.post
y_val = df.tags


In [15]:
flename_LOGREG = 'logreg.sav'
filename_SDG = 'SDG.sav'
flename_LOGREG2 = 'logreg_tuned.sav'
filename_SDG2 = 'SDG_tuned.sav'


In [18]:
loaded_model_LOGREG = pickle.load(open(flename_LOGREG2, 'rb'))
result_LOGREG = loaded_model_LOGREG.score(X_val,y_val)
print(result_LOGREG)
y_pred_LOGREG = loaded_model_LOGREG.predict(X_val)

0.8957345971563981


In [20]:
type(result_LOGREG)

numpy.float64

In [16]:
loaded_model_SDG = pickle.load(open(filename_SDG2, 'rb'))
result_SDG = loaded_model_SDG.score(X_val,y_val)
print(result_SDG)

y_pred_SDG = loaded_model_SDG.predict(X_val)

0.745260663507109


In [25]:
if 'result_LOGREG' < 'result_SDG':
    df['Predicted Class'] = y_pred_LOGREG
    df.to_csv('LOGREG_Results.csv')
    print('LOGREG SCORES HIGHER')
else:
    df['Predicted Class'] = y_pred_SDG
    df.to_csv('SDG_Results.csv')
    print('SDG SCORES HIGHER')


LOGREG SCORES HIGHER
